In [1]:
# designed to be run after 03-clinical_variables_final. this notebook does some data cleaning/processing. run before -___ notebook.

## cleans many aspects of the raw clinical variables.
## collapses and formats all of the various categorical variables into discrete variables as well. 

In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob
import sys

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.44 ms


In [5]:
sys.path.append('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/notebooks/')
sys.path.append('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/notebooks/parameters.py')
sys.path.append('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/notebooks/Exploratory/')

time: 673 µs


In [77]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v.copy()
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]


time: 15.1 ms


In [7]:
final_pt_df2['final_bin'].value_counts()

C_neg/A_partial    7867
C_neg/A_full       7401
C_pos/A_full       2438
C_pos/A_partial    1927
Name: final_bin, dtype: int64

time: 9.04 ms


# extracting clinical data for our patients
## IMPORTANT, USE THIS TO TUNE TIMEWINDOW OF EXTRACTION AND FOLDER TO SAVE IN

#NOTE ON MY DF NAMING CONVENTION:
origionally when I coded this workbook, it was for 72 hour timewindows, so every dataframe had _72 at the end. this was changed on 6/5/19 and was made more generalizable by finding name of each corresponding df in the df list and using this variable.

In [11]:
from parameters import date, time_col, time_var, patient_df, save_boolean
folder='30_day_window'
lower_window=0
upper_window=30

time: 574 µs


### begin pipeline:

# changing my code structure to be a dictionary of dataframes

In [12]:
#folder to save files to:
save_path= str(repository_path)+'/data/cleaned/'
#folder=None


def save_df(df, df_name='default', save_path=save_path, add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_prepped.csv' %(date, df_name)))


time: 1.17 ms


In [13]:
save_path+'%s/'%(folder)

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/30_day_window/'

time: 1.74 ms


In [14]:
##folder with all clinical variable csv's
allFiles = glob.glob(str(repository_path)+ '/data/raw/%s/'%(folder) + "{}_*.csv".format(date))

time: 1.27 ms


In [15]:
allFiles

['/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/30_day_window/16122019_sofa.csv']

time: 1.63 ms


In [16]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

time: 452 ms


In [17]:
df_list

['sofa']

time: 1.52 ms


In [19]:
#assigning the appropriate name to each df in a flexible way

indices = [i for i, s in enumerate(df_list) if 'sofa' in s]
sofa_df= df_list[indices[0]]


time: 607 µs


In [20]:
#adding a t_0 to each df that doesn't currently have it

for element in df_list:
    #print(element,':',list(dfs[element]))
    if ('t_0' in list(dfs[element]))==False and 'icustay_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['icustay_id','t_0']], how='left')
    elif ('t_0' in list(dfs[element]))==False and 'hadm_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['hadm_id','t_0']], how='left')
    else:
        print("false")


false
time: 1.6 ms


In [21]:
for element in df_list:
    print(element,':',list(dfs[element]))

sofa : ['subject_id', 'hadm_id', 'icustay_id', 'day', 'sofa', 'respiration', 'pao2fio2_vent_min', 'pao2fio2_novent_min', 'coagulation', 'platelet_min', 'liver', 'bilirubin_max', 'cardiovascular', 'rate_dopamine', 'rate_epinephrine', 'rate_norepinephrine', 'rate_dobutamine', 'meanbp_min', 'cns', 'mingcs', 'renal', 'creatinine_max', 'urineoutput', 't_0', 'icu_admit', 'approx_charttime', 'floor_charttime', 'floor_time_var']
time: 661 µs


In [22]:
from parameters import time_var, value_fill, delta_fill, uom_fill
def yn_convert(df, #df in format where each row corresponds to a test, and a patient can have many rows
               label_fill, # value that will be filled to na's
               pt= final_pt_df2,
               time_var=time_var,#'t_0', #
               value_fill=value_fill,#0,
               delta_fill=delta_fill,# pd.to_timedelta('0 days'),
               uom_fill=uom_fill):#'y/n'):

    """
    description: collapses (binarizes) a dataframe where each row corresponds to a test, and a patient can have many rows -> 
    1 row per patient where value is binary variable yes or no a patient has any value within the timewindow (specified in data collection).
    said a different way, for patient this fxn collapses values down to does pt have a non NA value in the clinical time window y/n? 
    
    label_fill: the variable name in the label column of the specified dataframe that will considered for y/n value within timewindow. if any non NA value is present
        it will be considered positive.
    pt: the by patient spreadsheet be be used to supply patient information.
    time_var: the variable used to create the time window of interest.
    value_fill: the variable value that missing values will be filled if the value is not present (default =0) in the origional dataset
    delta_fill: the time delta value that will be filled in if a patient doesn't have any instances of the label_fill.  
    uom_fill: fills in the unit of measurement to this for missing values.
    
    returns a flat 1 row per icustay_id of 1 or 0 if any value was present for the patient.
    """
    
    yn_df = pd.merge(pt[['icustay_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging all icustay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    yn_df.loc[yn_df.loc[:,'value']!=value_fill, 'value']= 1 #squashes all other values into a binary 1 = yes
    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    ##this is new as of 12-12-19L i think i wasn't truely converting to 1row per icustay id by not droping duplicates filtered on value first (desc) then delta (asc)
    yn_df= yn_df.sort_values(['value','delta'], ascending=[False, True]).drop_duplicates(subset='icustay_id',keep='first')
    
    return(yn_df)


time: 2.52 ms


# SOFA
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.
also added delta

In [23]:
dfs[sofa_df]['uom']='daily_sofa_score'

#adding day delta column
dfs[sofa_df]=dfs[sofa_df].sort_values(['hadm_id','day',time_var]) #good
dfs[sofa_df]['day_rank']=dfs[sofa_df].groupby('icustay_id')['day'].rank()
dfs[sofa_df]['delta']=pd.to_timedelta((dfs[sofa_df]['day_rank']-1), 'days')

time: 205 ms


In [24]:
dfs[sofa_df]['label']= 'daily_sofa'
dfs[sofa_df]=dfs[sofa_df].rename(index=str, columns={'sofa':'value'})

time: 291 ms


In [25]:
dfs[sofa_df]= dfs[sofa_df][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 33.1 ms


# now to code organ dysfunction free days:

algorithm:
* sum # of days where daily sofa <1 and a person is alive starting at t_0 and ending at t_0+30.

* If the leave the ICU before 30 days and there is no data and they are not dead, you assume their SOFA is 0 until day 30.

In [48]:
final_pt_df2.head()

,icustay_id,ssc_charttime,ssc_id,icu_admit,ab_id,ab_start,ab_end,ab_ssc_delta,t_0,t_end,...,dod_ssn,ab_course,org_list,spec_type_list,first_pos_else_neg_ssc,sc_result,final_bin,hadm_id,delta,dod
0,294638,2191-03-16 00:00:00,22,2191-03-16 00:29:31,1213116,2191-03-16,2191-03-22,0 days 00:00:00.000000000,2191-03-16,2191-03-22,...,NaT,full,STAPH AUREUS COAG +,BLOOD CULTURE,2191-03-16 00:00:00,positive,C_pos/A_full,185777,NaT,NaT
1,220597,2149-11-10 09:40:00,45,2149-11-09 13:07:02,2089126,2149-11-10,2149-11-15,0 days 00:00:00.000000000,2149-11-10,2149-11-15,...,2149-11-14,full,NaN,NaN,2149-11-10 09:40:00,negative,C_neg/A_full,150750,0 days,2149-11-14
2,232669,2104-08-11 00:00:00,60,2104-08-08 02:08:17,616189,2104-08-11,2104-08-12,0 days 00:00:00.000000000,2104-08-11,2104-08-12,...,2104-08-20,partial,NaN,NaN,2104-08-11 00:00:00,negative,C_neg/A_partial,112213,0 days,2104-08-20
3,273430,2108-08-05 20:42:00,69,2108-08-05 16:26:09,2572274,2108-08-06,2108-08-08,1 days 00:00:00.000000000,2108-08-06,2108-08-08,...,2109-08-18,partial,NaN,NaN,2108-08-05 20:42:00,negative,C_neg/A_partial,109235,NaT,2109-08-18
4,217847,2134-09-11 09:35:00,70,2134-09-11 20:50:04,1388217,2134-09-12,2134-09-13,1 days 00:00:00.000000000,2134-09-12,2134-09-13,...,2135-02-08,full,NaN,NaN,2134-09-11 09:35:00,negative,C_neg/A_full,109451,0 days,2135-02-08


time: 23 ms


In [34]:
from datetime import timedelta

time: 461 µs


In [ ]:
# ssc_t0ab_t0ssc.loc[
#     ((pd.to_datetime(ssc_t0ab_t0ssc['t_end_consec'])+ pd.DateOffset(1)) 
#      >= pd.to_datetime(ssc_t0ab_t0ssc['dod_hosp'])),
#     'ab_course']='full'

In [78]:
#dod col is now the min date of death between hospital and ssn (about 150 discrepencies)
final_pt_df2['dod_hosp']=pd.to_datetime(final_pt_df2['dod_hosp'])
final_pt_df2['dod_ssn']=pd.to_datetime(final_pt_df2['dod_ssn'])

final_pt_df2['dod']=pd.to_datetime(final_pt_df2[['dod_hosp','dod_ssn']].min(axis=1))

#making a delta col between dod and t_0
final_pt_df2['dod_t0_delta']= (pd.to_datetime(final_pt_df2['dod_hosp'])-
                               pd.to_datetime(final_pt_df2['t_0']))


time: 15.2 ms


In [90]:
final_pt_df2['dod_t0_delta'].value_counts().reset_index()['index'].sort_values()





595      -1 days
2         0 days
0         1 days
1         2 days
3         3 days
          ...   
760    3304 days
865    3313 days
1023   3357 days
1190   3420 days
925    3535 days
Name: index, Length: 1299, dtype: timedelta64[ns]

time: 7.41 ms


In [108]:
print(final_pt_df2.loc[final_pt_df2['dod_t0_delta']=='-1 days',['dod_hosp','dod_ssn','dod','t_0','t_end']])


        dod_hosp    dod_ssn        dod         t_0       t_end
13767 2165-02-06 2165-02-06 2165-02-06  2165-02-07  2165-02-08
15075 2190-12-10 2190-12-10 2190-12-10  2190-12-11  2190-12-12
time: 12.3 ms


In [107]:
final_pt_df2.loc[final_pt_df2['dod_t0_delta']=='-1 days','icustay_id']

13767    209071
15075    281048
Name: icustay_id, dtype: int64

time: 3.8 ms


### found weird result: two patints have t_0 AFTER dod, need to explore??

subject_id= 57518,66774
hadm_id=105024, 199087
icustay_id= 209071, 281048

        dod_hosp    dod_ssn        dod         t_0
13767 2165-02-06 2165-02-06 2165-02-06  2165-02-07
15075 2190-12-10 2190-12-10 2190-12-10  2190-12-11

In [104]:
list(final_pt_df2)

['icustay_id',
 'ssc_charttime',
 'ssc_id',
 'icu_admit',
 'ab_id',
 'ab_start',
 'ab_end',
 'ab_ssc_delta',
 't_0',
 't_end',
 't_0_sc',
 'abduration',
 't_end_consec',
 'subject_id',
 'dod_hosp',
 'dod_ssn',
 'ab_course',
 'org_list',
 'spec_type_list',
 'first_pos_else_neg_ssc',
 'sc_result',
 'final_bin',
 'hadm_id',
 'delta',
 'dod',
 'dod_t0_delta']

time: 2.46 ms


In [71]:
pd.to_datetime(0)

Timestamp('1970-01-01 00:00:00')

time: 2.34 ms


In [82]:
#final_pt_df2['dod_hosp']- 
final_pt_df2['dod_t0_delta']= pd.to_datetime(final_pt_df2['dod_hosp'])- pd.to_datetime(final_pt_df2['t_0'])


time: 8.4 ms


In [55]:
final_pt_df2['dod_hosp']

0                          0
1        2149-11-14 00:00:00
2        2104-08-20 00:00:00
3                          0
4        2135-02-08 00:00:00
                ...         
19628                      0
19629                      0
19630                      0
19631                      0
19632                      0
Name: dod_hosp, Length: 19633, dtype: object

time: 3.77 ms


In [26]:
dfs[sofa_df].head()

,subject_id,hadm_id,icustay_id,delta,label,value,t_0,uom
88189,54610,100003,209281,0 days,daily_sofa,4,2150-04-18,daily_sofa_score
52007,23018,100007,217937,0 days,daily_sofa,1,2145-03-31,daily_sofa_score
52008,23018,100007,217937,1 days,daily_sofa,4,2145-03-31,daily_sofa_score
52009,23018,100007,217937,2 days,daily_sofa,0,2145-03-31,daily_sofa_score
52010,23018,100007,217937,3 days,daily_sofa,3,2145-03-31,daily_sofa_score


time: 14.8 ms


In [29]:
dfs[sofa_df]['delta']

88189   0 days
52007   0 days
52008   1 days
52009   2 days
52010   3 days
         ...  
58368   0 days
58369   1 days
44337   0 days
44338   1 days
62076   0 days
Name: delta, Length: 106097, dtype: timedelta64[ns]

time: 26 ms


In [132]:
save_df(dfs[sofa_df], 'sofa')
del(dfs[sofa_df])

time: 2.23 s
